In [37]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.test_suite import TestSuite
from checklist.expect import Expect

In [ ]:
from tqdm import tqmd
from torch.utils.data import DataLoader

In [87]:
import sys
import re
import spacy
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
np.random.seed(172)
import pandas as pd
from tqdm import tqdm
processor = spacy.load('en_core_web_sm')

In [79]:
df_train = pd.read_csv('data/processed/train.csv')
df_train = df_train[df_train['sentiment']!='neutral']
text = df_train['text'].tolist()
y_trues = df_train['sentiment'].str.upper()

In [36]:
import transformers
pipe = transformers.pipeline('sentiment-analysis', device=0)

In [99]:
def predict_proba(X):
    preds = []
    loader = DataLoader(X, batch_size=64, shuffle=False)
    for batch in loader:
        batch_preds = pipe.predict(batch)
        preds += batch_preds
    preds = [(x['label'], x['score']) for x in preds]
    # rval = np.zeros(len(preds))
    rval = list(map(lambda x: x[1] if x[0] == 'POSITIVE' else 1-x[1], preds))
    rval = np.array(rval).reshape(-1,1)
    rval = np.hstack((1-rval, rval))
    # rval = np.
    # preds, scores = list(zip(*batch_preds))
    # preds = [x[1] for x in preds]
    return rval

predict_proba(['good', 'great', 'terrible'])

array([[ 0.0002,  0.9998],
       [ 0.0001,  0.9999],
       [ 0.9998,  0.0002]])

In [100]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)
wrapped_pp(['good', 'great', 'terrible'])

(array([1, 1, 0]),
 array([[ 0.0002,  0.9998],
        [ 0.0001,  0.9999],
        [ 0.9998,  0.0002]]))

In [56]:
data = list(tqdm(processor.pipe(text, batch_size=64)))

7285it [00:30, 236.13it/s]


In [12]:
suite = TestSuite()
editor = Editor()

In [13]:
# bottom up

In [18]:
np.random.choice(text, 5)

array(['@usairways aye and nothing to do with an automated system which doesn t work properly anyway got through to somebody ',
       '@southwestair done messed up now i have planned a trip to costarica for pennies vacatime ',
       '@usairways i m delayed in tys 5015 likely gonna miss my connection in clt to cun gotta get there today can you help me ',
       '@united direct messaged you didn t hear anything back',
       '@united thanks for making everyone wait in the dark for an hour while newark tries to figure out which gate to go to incompetence'],
      dtype='<U177')

In [25]:
def change_airline(x, *args, **kwargs):
    # Returns empty or a list of strings with profesions changed
    airlines = ['jetblue', 'virginamerica', 'united', 'southwestair', 'usairways', 'americanair']
    ret = []
    for p in airlines:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in airlines if p != p2])
    return ret

change_airline(text[3])

['@jetblue flight home cancelled flightled today any way we can get another flight to nyc from texas 4 hour wait for a call back from airline',
 '@virginamerica flight home cancelled flightled today any way we can get another flight to nyc from texas 4 hour wait for a call back from airline',
 '@united flight home cancelled flightled today any way we can get another flight to nyc from texas 4 hour wait for a call back from airline',
 '@southwestair flight home cancelled flightled today any way we can get another flight to nyc from texas 4 hour wait for a call back from airline',
 '@usairways flight home cancelled flightled today any way we can get another flight to nyc from texas 4 hour wait for a call back from airline']

In [104]:
change_airline_perturb = Perturb.perturb(text, change_airline, keep_original=True)
change_airline_test= INV(**change_airline_perturb)
change_airline_test.data[1]

['@united have better customer service at john wayne airport ',
 '@jetblue have better customer service at john wayne airport ',
 '@virginamerica have better customer service at john wayne airport ',
 '@southwestair have better customer service at john wayne airport ',
 '@usairways have better customer service at john wayne airport ',
 '@americanair have better customer service at john wayne airport ']

In [105]:
change_airline_test.run(wrapped_pp)

Predicting 44714 examples


KeyboardInterrupt: 